In [11]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [12]:
genre_id = "34399"
from_ix = 0
to_ix = 10

In [13]:
def get_titles_info(title_id: str):
    title_info_dict = {
        "title": "",
        "overview": "",
        "genres": [],
        "cast": [],
        "content_is": "",
        "subtitles": [],
        "audio": [],
    }

    request_url = f"https://www.netflix.com/title/{title_id}"
    response = requests.get(
        request_url
    )
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    if soup:
        title_info_dict.update({
            "title": soup.find("h1", class_="title-title").text,
            "overview": soup.find("div", class_="title-info-synopsis").text,
            "genres": [spn.text.replace(',','').strip() for spn in soup.find("div", "more-details-cell cell-genres").findAll('span')], 
            "cast": [spn.text.replace(',','').strip() for spn in soup.find("div", "more-details-cell cell-cast").findAll('span')]
        })
    
        content_mood_div = soup.find("div", "more-details-cell cell-mood-tag")
        if content_mood_div:
            title_info_dict.update({
                "content_is": [spn.text.replace(',','').strip() for spn in content_mood_div.findAll('span')],   
            })


        subtitles_div = soup.find("div", "more-details-cell cell-subtitle")
        if subtitles_div:
            title_info_dict.update({
                "subtitles": [spn.text.replace(',','').strip() for spn in subtitles_div.findAll('span')],   
            })
        
        audio_div = soup.find("div", "more-details-cell cell-audio")
        if subtitles_div:
            title_info_dict.update({
                "audio": [spn.text.replace(',','').strip() for spn in audio_div.findAll('span')],   
            })

    return title_info_dict

In [14]:
cookies = {
    'SecureNetflixId': '<your_netflix_id>',
    'NetflixId': '<your_netflix_id>',
    'profilesNewSession': '0'
}

headers = {
    'authority': 'www.netflix.com',
    'accept': '*/*',
    'accept-language': 'en-GB,en;q=0.7',
    'content-type': 'application/x-www-form-urlencoded',
    'origin': 'https://www.netflix.com',
    'referer': 'https://www.netflix.com/browse/genre/34399?so=az',
    'sec-ch-ua': '"Brave";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"macOS"',
    'sec-ch-ua-platform-version': '"13.6.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'x-netflix.browsername': 'Chrome',
    'x-netflix.browserversion': '117',
    'x-netflix.client.request.name': 'ui/falcorUnclassified',
    'x-netflix.clienttype': 'akira',
    'x-netflix.esn': 'NFCDCH-MC-KJA5N1EWHGG3RYLYDJ4X0UC97AN6JG',
    'x-netflix.esnprefix': 'NFCDCH-MC-',
    'x-netflix.nq.stack': 'prod',
    'x-netflix.osfullname': 'Mac OS X',
    'x-netflix.osname': 'Mac OS X',
    'x-netflix.osversion': '10.15.7',
    'x-netflix.request.client.user.guid': 'WSAZV47GUFD4HITW5K7QSDLNPA',
    'x-netflix.uiversion': 'vc63e5850',
}

params = {
    'webp': 'true',
    'drmSystem': 'widevine',
    'isVolatileBillboardsEnabled': 'true',
    'routeAPIRequestsThroughFTL': 'false',
    'hasVideoMerchInBob': 'true',
    'hasVideoMerchInJaw': 'true',
    'falcor_server': '0.1.0',
    'withSize': 'true',
    'materialize': 'true',
    'original_path': '/shakti/mre/pathEvaluator',
}


genre_item_summ_path = f'["genres",{genre_id},"az",{{"from":{from_ix},"to":{to_ix}}},"itemSummary"]'
genre_item_ref_summ_path = f'["genres",{genre_id},"az",{{"from":{from_ix},"to":{to_ix}}},"reference",["availability","episodeCount","inRemindMeList","queue","summary"]]'
data = {
    'path': [
        genre_item_summ_path,
        genre_item_ref_summ_path,
    ],
    'authURL': '1697060641282.S8RDZDjKsQ5wO5euqI+1oE0e+Eg=',
}

response = requests.post(
    'https://www.netflix.com/nq/website/memberapi/vc63e5850/pathEvaluator',
    params=params,
    cookies=cookies,
    headers=headers,
    data=data,
)


print(f"fetch status code --> {response.status_code}")
# json.dumps(response.json())

fetch status code --> 200


In [21]:
if response.status_code == 200:
    summary_data = response.json()
    title_info_list = []
    
    for ix, item_summ in summary_data["jsonGraph"]["genres"][genre_id]['az'].items():
        title_info_dict = {}
        
        if not item_summ["itemSummary"]["value"]:
            break
    
        item_summ_val = item_summ["itemSummary"]["value"]

        print(f"processing for title_id -> {item_summ_val['id']}")
        
        misc_title_info_dict = get_titles_info(str(item_summ_val["id"]))        
        
        title_info_dict.update({
            "id": str(item_summ_val["id"]),
            "title": item_summ_val["title"],
            "type": item_summ_val["type"],
            "release_year": item_summ_val["releaseYear"],
            "runtime": item_summ_val["infoDensityRuntime"],
            "backdrop_path": item_summ_val["boxArt"]["url"],
            "made_by_netflix": item_summ_val["isOriginal"],
            "maturity_rating": item_summ_val["maturity"]["rating"]["value"],
            "maturity_warning": item_summ_val["maturity"]["rating"]["maturityDescription"],
            "maturity_rating_reason": [ix for ix in item_summ_val["maturity"]["rating"]["specificRatingReason"].split(', ')],
            "maturity_rating_board": item_summ_val["maturity"]["rating"]["board"], 
        })

        title_info_dict = {**title_info_dict, **misc_title_info_dict}
        
        title_info_list.append(title_info_dict)

processing for title_id -> 80997613
processing for title_id -> 81357618
processing for title_id -> 81412227
processing for title_id -> 81144147
processing for title_id -> 81644889
processing for title_id -> 81240831
processing for title_id -> 81260630
processing for title_id -> 81260637
processing for title_id -> 80125979
processing for title_id -> 81092768
processing for title_id -> 81092766


In [26]:
pd.DataFrame.from_records(title_info_list)

,id,title,type,release_year,runtime,backdrop_path,made_by_netflix,maturity_rating,maturity_warning,maturity_rating_reason,maturity_rating_board,overview,genres,cast,content_is,subtitles,audio
0,80997613,"¡Ay, mi madre!",movie,2019,4884,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,False,U/A 16+,Suitable for persons aged 16 and above and und...,"[language, tobacco use]",India,"When her estranged mother suddenly dies, a wom...","[Spanish, Comedy Movies]","[Estefanía de los Santos, Secun de la Rosa, Te...","[Sentimental, Intimate]","[English, European Spanish, Hindi]","[European Spanish - Audio Description, Europea..."
1,81357618,¡Que viva México!,movie,2023,11502,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,True,A,Content restricted to adults,"[nudity, sex, tobacco use]",India,"After his miner grandfather's death, Pancho tr...","[Mexican, Satires, Dramas, Comedy Movies, Soci...","[Damián Alcázar, Alfonso Herrera, Joaquín Cosí...",[Provocative],"[English, English, Spanish]","[Spanish - Audio Description, Spanish [Original]]"
2,81412227,'76,movie,2016,7084,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,True,U/A 13+,Suitable for persons aged 13 and above and und...,"[gore, mature themes, tobacco use, violence]",India,When her husband is accused of taking part in ...,"[Nollywood, Military Movies, Dramas, African M...","[Ramsey Nouah, Rita Dominic, Chidi Mokeme, Ibi...",[Emotional],[English],"[English - Audio Description, English [Original]]"
3,81144147,'83,movie,2021,9626,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,False,U/A 13+,Suitable for persons aged 13 and above and und...,"[crude humor, gore, language, tobacco use, vio...",India,"Amid doubt and derision, indomitable captain K...","[Hindi-Language Movies, Sports Movies, Bollywo...","[Ranveer Singh, Deepika Padukone, Pankaj Tripa...","[Inspiring, Heartfelt]","[English, Hindi]","[Hindi [Original], Kannada, Malayalam, Tamil, ..."
4,81644889,"""Sr.""",movie,2022,5364,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,True,A,Content restricted to adults,"[substances, tobacco use]",India,Robert Downey Jr. pays tribute to his late fat...,"[Documentaries, Biographical Documentaries, US...","[Robert Downey Sr., Robert Downey Jr.]","[Intimate, Emotional]",[English],"[English - Audio Description, English [Original]]"
5,81240831,#Alive,movie,2020,5914,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,True,U/A 16+,Suitable for persons aged 16 and above and und...,"[gore, language, suicide, tobacco use, violence]",India,"As a grisly virus rampages a city, a lone man ...","[Korean, Horror Movies]","[Yoo Ah-in, Park Shin-hye]",,"[English, English, Korean]","[English, Korean - Audio Description, Korean [..."
6,81260630,#FriendButMarried,movie,2018,6125,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,False,U/A 7+,Suitable for children 7 and above and under pa...,[mild language],India,"Pining for his high school crush for years, a ...","[Indonesian, Dramas, Romantic Movies, Movies B...","[Adipati Dolken, Vanesha Prescilla, Rendi Jhon...","[Swoonworthy, Sentimental, Bittersweet]","[English, Indonesian]",[Indonesian [Original]]
7,81260637,#FriendButMarried 2,movie,2020,6246,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,False,U/A 7+,Suitable for children 7 and above and under pa...,"[mild language, mild violence]",India,As Ayu and Ditto finally transition from best ...,"[Indonesian, Dramas, Romantic Movies, Movies B...","[Adipati Dolken, Mawar de Jongh, Sari Nila, Vo...","[Bittersweet, Emotional, Romantic]","[English, Indonesian]",[Indonesian [Original]]
8,80125979,#realityhigh,movie,2017,5965,https://occ-0-2890-2186.1.nflxso.net/dnm/api/v...,True,U/A 16+,Suitable for persons aged 16 and above and und...,"[crude humor, language]",India,When nerdy high schooler Dani finally attracts...,"[Comedy Movies, Teen Movies, US Movies]","[Nesta Cooper, Kate Walsh, John Michael Higgin...","[Witty, Heartfelt, Feel-Good]",[English],"[English - Audio Description, English [Original]]"
9,81092